# Importações

In [2]:
import pandas as pd

# 1. Métricas Monitorar com ``iqar`` e presentes no DataSUS

## 1. Tratamento de Dados

### 1.1 MonitorAr

#### Lê os ``.csv``s

Estações

In [3]:
stations_cols=[
    'Código IBGE do Município', 'Nome do Município', 'Estado', 'Nome da Estação'
]
        
stations_df = pd.read_csv("../data/monitor_ar/EstacoesMonitorAr-Nov-2022.csv", sep=";", usecols=stations_cols)

stations_df.rename(columns = {
    "Código IBGE do Município": "Codigo_IBGE",
    "Nome do Município": "Nome_Municipio",
    "Nome da Estação": "Nome_Estacao"
}, inplace=True)

stations_df.rename(columns = {
    "Código IBGE do Município": "Codigo_IBGE"
}, inplace=True)

stations_df['Codigo_IBGE'] = (stations_df['Codigo_IBGE'] / 10).astype(int)

Monitores

In [4]:
def get_monitors(cols=None):
    jan_mar_df = pd.read_csv("../data/monitor_ar/Dados_monitorar_jan_mar.csv", encoding="latin", sep=";", usecols=cols)
    abr_jun_df = pd.read_csv("../data/monitor_ar/Dados_monitorar_abr_jun.csv", encoding="latin", sep=";", usecols=cols)
    jul_nov_df = pd.read_csv("../data/monitor_ar/Dados_monitorar_jul_nov.csv", encoding="latin", sep=";", usecols=cols)

    # Junta as três tabelas de monitores
    monitors_df = pd.concat([jan_mar_df, abr_jun_df, jul_nov_df], axis=0)

    return monitors_df

monitors_cols=[
    'Nome do Município', 'Estado', 'Nome da Estação', 'Sigla', 'Concentracao', 'iqar', 'Data'
]
monitors_df = get_monitors(monitors_cols)

monitors_df.rename(columns = {
    "Nome do Município": "Nome_Municipio",
    "Nome da Estação": "Nome_Estacao"
}, inplace=True)

C:\Users\Aluno\AppData\Local\Temp\ipykernel_13832\4180103939.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  jul_nov_df = pd.read_csv("../data/monitor_ar/Dados_monitorar_jul_nov.csv", encoding="latin", sep=";", usecols=cols)


#### 2. Realizar o ``.merge`` (``JOIN``) entre os Dataframes

In [5]:
monitorar_df = pd.merge(
    left=monitors_df,
    right=stations_df,
    on=['Nome_Municipio', 'Nome_Estacao', 'Estado'],
    how='inner'
)

monitorar_df['iqar'] = pd.to_numeric(monitorar_df['iqar'], errors='coerce')

all = monitorar_df.groupby('Estado').size()
print(f"Todos os registros")
print(all.sort_values())


monitorar_df = monitorar_df.dropna(subset=['iqar'])
monitorar_df = monitorar_df.sort_values(by='iqar')

print("-="*30)

filtered = monitorar_df.groupby('Estado').size()
print(f"Filtrados")
print(filtered.sort_values())

Todos os registros
Estado
PR     18627
RS     66649
MS     85804
ES    115858
SP    425815
MG    454806
RJ    520817
BA    611857
dtype: int64
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Filtrados
Estado
PR     15058
MS     27719
RS     60000
ES    115858
RJ    134483
BA    187675
MG    288378
SP    425815
dtype: int64


### 1.2 DataSUS

#### Lê o ``.csv``

In [6]:
cols = [
    'CO_MUN_RES'
]
    
datasus_df = pd.read_csv("../data/data_sus/INFLUD22-26-06-2025.csv", sep=";", usecols=cols)

municipios_code = datasus_df.rename(columns={
    "CO_MUN_RES": "Codigo_IBGE"
})

municipios_code.dropna(subset=['Codigo_IBGE'], inplace=True)
municipios_code["Codigo_IBGE"] = municipios_code["Codigo_IBGE"].astype(int)
municipios_code.drop_duplicates(subset=['Codigo_IBGE'])
municipios_code = municipios_code['Codigo_IBGE']

## 2. ``Filter`` entre Códigos IBGE no ``DataSUS`` e Métricas com ``iqar``

In [7]:
final_df = monitorar_df[monitorar_df['Codigo_IBGE'].isin(list(municipios_code))]
datasus_filtered = final_df.groupby('Estado').size()

print(f"Filtrados e Presentes no DATASUS")
print(datasus_filtered.sort_values())

Filtrados e Presentes no DATASUS
Estado
PR     15058
MS     27719
RS     60000
ES    115858
RJ    134483
BA    187675
MG    288378
SP    425815
dtype: int64


# 2. Casos presentes no DataSUS e com ``iqar`` presente no MonitorAr

## 1. Tratamento de Dados

### 1.1 MonitorAr

##### Lê os ``.csv``s

Estações

In [8]:
stations_cols=[
    'Código IBGE do Município', 'Nome do Município', 'Estado', 'Nome da Estação'
]
        
stations_df = pd.read_csv("../data/monitor_ar/EstacoesMonitorAr-Nov-2022.csv", sep=";", usecols=stations_cols)

stations_df.rename(columns = {
    "Código IBGE do Município": "Codigo_IBGE",
    "Nome do Município": "Nome_Municipio",
    "Nome da Estação": "Nome_Estacao"
}, inplace=True)

stations_df.rename(columns = {
    "Código IBGE do Município": "Codigo_IBGE"
}, inplace=True)

stations_df['Codigo_IBGE'] = (stations_df['Codigo_IBGE'] / 10).astype(int)

Monitores

In [9]:
def get_monitors(cols=None):
    jan_mar_df = pd.read_csv("../data/monitor_ar/Dados_monitorar_jan_mar.csv", encoding="latin", sep=";", usecols=cols)
    abr_jun_df = pd.read_csv("../data/monitor_ar/Dados_monitorar_abr_jun.csv", encoding="latin", sep=";", usecols=cols)
    jul_nov_df = pd.read_csv("../data/monitor_ar/Dados_monitorar_jul_nov.csv", encoding="latin", sep=";", usecols=cols)

    # Junta as três tabelas de monitores
    monitors_df = pd.concat([jan_mar_df, abr_jun_df, jul_nov_df], axis=0)

    return monitors_df

monitors_cols=[
    'Nome do Município', 'Estado', 'Nome da Estação', 'Sigla', 'Concentracao', 'iqar', 'Data'
]
monitors_df = get_monitors(monitors_cols)

monitors_df.rename(columns = {
    "Nome do Município": "Nome_Municipio",
    "Nome da Estação": "Nome_Estacao"
}, inplace=True)

C:\Users\Aluno\AppData\Local\Temp\ipykernel_13832\4180103939.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  jul_nov_df = pd.read_csv("../data/monitor_ar/Dados_monitorar_jul_nov.csv", encoding="latin", sep=";", usecols=cols)


##### Realizar o ``.merge`` (``JOIN``) entre os Dataframes

In [10]:
monitorar_df = pd.merge(
    left=monitors_df,
    right=stations_df,
    on=['Nome_Municipio', 'Nome_Estacao', 'Estado'],
    how='inner'
)

monitorar_df['iqar'] = pd.to_numeric(monitorar_df['iqar'], errors='coerce')
monitorar_df = monitorar_df.dropna(subset=['iqar'])
monitorar_df = monitorar_df.sort_values(by='iqar')
ibge_codes = monitorar_df['Codigo_IBGE'].drop_duplicates()
ibge_codes

1716312    330455
1926218    500830
1080868    310670
1579766    410690
1894742    432200
            ...  
660749     350320
1801763    350160
591160     315120
442748     311940
1914839    352940
Name: Codigo_IBGE, Length: 72, dtype: int64

### 1.2 DataSUS

##### Lê o ``.csv``

In [11]:
cols = [
    'CO_MUN_RES', 'SG_UF', 'ID_MN_RESI'
]
    
datasus_df = pd.read_csv("../data/data_sus/INFLUD22-26-06-2025.csv", sep=";", usecols=cols)

datasus_df = datasus_df.rename(columns={
    "CO_MUN_RES": "Codigo_IBGE",
    "SG_UF": "Estado",
    "ID_MN_RESI": "Municipio"
})

datasus_df

,Estado,Municipio,Codigo_IBGE
0,RJ,SAO GONCALO,330490.0
1,SP,SAO CAETANO DO SUL,354880.0
2,SP,SAO CAETANO DO SUL,354880.0
3,CE,JUAZEIRO DO NORTE,230730.0
4,CE,JUAZEIRO DO NORTE,230730.0
...,...,...,...
560572,MG,CAMPO BELO,311120.0
560573,MG,CONTAGEM,311860.0
560574,MG,BELO HORIZONTE,310620.0
560575,SE,LAGARTO,280350.0


## 2. ``Filter`` entre Códigos IBGE no ``Monitorar`` (com ``iqar``) e casos no ``DataSUS``

In [ ]:
final_df = datasus_df[datasus_df['Codigo_IBGE'].isin(list(ibge_codes))]
datasus_filtered = final_df.groupby('Estado').size()

print(f"Filtrados e Presentes no MonitorAr")
print(datasus_filtered.sort_values())

560577
158989
Filtrados e Presentes no MonitorAr
Estado
MS       407
BA       636
ES      2786
RS      2834
PR     12912
RJ     14914
MG     21603
SP    102897
dtype: int64
